In [1]:
import numpy as np
import pandas as pd

In [2]:
df_sample=pd.read_csv(r"C:\Users\23059\OneDrive\Desktop\Amiira\Y3S1\fyp\sample.csv")

In [3]:
df_sample.shape

(700000, 11)

In [4]:
# Change the data type of column 'A' from float64 to float32
df_sample['amount'] = df_sample['amount'].astype('float32')
df_sample['oldbalanceOrg'] = df_sample['oldbalanceOrg'].astype('float32')
df_sample['oldbalanceDest'] = df_sample['oldbalanceDest'].astype('float32')
df_sample['newbalanceOrig'] = df_sample['newbalanceOrig'].astype('float32')
df_sample['newbalanceDest'] = df_sample['newbalanceDest'].astype('float32')

In [5]:
df_sample['step'] = df_sample['step'].astype('int32')
df_sample['isFlaggedFraud'] = df_sample['isFlaggedFraud'].astype('int32') 
df_sample['isFraud'] = df_sample['isFraud'].astype('int32') 

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(df_sample['type'])
label
df_sample.drop("type", axis=1, inplace=True)
df_sample["type"] = label

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(df_sample['nameDest'])
label
df_sample.drop("nameDest", axis=1, inplace=True)
df_sample["nameDest"] = label

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(df_sample['nameOrig'])
label
df_sample.drop("nameOrig", axis=1, inplace=True)
df_sample["nameOrig"] = label

In [9]:
from sklearn.model_selection import train_test_split

X = df_sample.drop('isFraud', axis=1)
# Separate the target variable
y = df_sample['isFraud']

# Print class distribution before split
print(y.value_counts(normalize=True))

from sklearn.model_selection import train_test_split

# Assuming X contains your features and y contains your target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=2)

# Print class distribution after split
print(y_train.value_counts(normalize=True))
print(y_test.value_counts(normalize=True))

0    0.998743
1    0.001257
Name: isFraud, dtype: float64
0    0.998743
1    0.001257
Name: isFraud, dtype: float64
0    0.998743
1    0.001257
Name: isFraud, dtype: float64


In [10]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# Set up the resampling methods
smote = SMOTE(sampling_strategy=0.2, random_state=0)
rus = RandomUnderSampler(sampling_strategy=0.4, random_state=0)

# Apply the resampling methods
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
X_resampled, y_resampled = rus.fit_resample(X_resampled, y_resampled)

In [11]:
import numpy as np

# assuming y_train is a numpy array or a pandas series
counts = np.bincount(y_resampled)
print("Class 0 count:", counts[0])
print("Class 1 count:", counts[1])

Class 0 count: 314602
Class 1 count: 125841


In [12]:
from imblearn.under_sampling import TomekLinks, EditedNearestNeighbours,OneSidedSelection

# Assume X_train and y_train are the original training data
# resample using TomekLinks first
tomek_links = TomekLinks(sampling_strategy='majority')
X_train_resampled, y_train_resampled = tomek_links.fit_resample(X_resampled, y_resampled)

In [13]:
import numpy as np

# assuming y_train is a numpy array or a pandas series
counts = np.bincount(y_train_resampled)
print("Class 0 count:", counts[0])
print("Class 1 count:", counts[1])

Class 0 count: 314594
Class 1 count: 125841


In [14]:
# resample the output of TomekLinks using EditedNearestNeighbours
enn = EditedNearestNeighbours(sampling_strategy='majority')
X_train_resampled_new, y_train_resampled_new = enn.fit_resample(X_train_resampled, y_train_resampled)

In [15]:
import numpy as np

# assuming y_train is a numpy array or a pandas series
counts = np.bincount(y_train_resampled_new)
print("Class 0 count:", counts[0])
print("Class 1 count:", counts[1])

Class 0 count: 311729
Class 1 count: 125841


In [16]:
from imblearn.under_sampling import OneSidedSelection
# resample the output of EditedNearestNeighbours using One-Sided Selection
oss = OneSidedSelection(sampling_strategy='majority')
X_train_resampled_final, y_train_resampled_final = oss.fit_resample(X_train_resampled_new, y_train_resampled_new)

In [17]:
import numpy as np

counts = np.bincount(y_train_resampled_final)
print("Class 0 count:", counts[0])
print("Class 1 count:", counts[1])

Class 0 count: 309811
Class 1 count: 125841


In [18]:
import random
import numpy as np
from scipy import stats

# Specify columns with outliers
cols_with_outliers = ['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']

# Specify the number of bootstrapped samples to create per column
num_samples = 50

# Specify the right trimming proportions for each column
trim_props = {'amount': 0.14, 'oldbalanceOrg': 0.24, 'newbalanceOrig': 0.21, 'oldbalanceDest': 0.2, 'newbalanceDest': 0.22}

# Initialize empty dictionaries to store the trimmed means for each column
train_trimmed_means = {}

# Loop over the specified columns
for col_name in cols_with_outliers:
    
    # Check if the trimming proportion for this column is 0
    if trim_props[col_name] == 0:
        # If so, skip this column and move on to the next one
        continue
    
    # Initialize empty lists to store the bootstrapped samples and the trimmed means for the training set
    train_bootstrapped_samples = []
    train_trimmed_means_list = []
    
    # Loop over the number of desired samples
    for i in range(num_samples):
        # Randomly select indices from the column in the training set
        train_sample_indices = np.random.choice(X_train_resampled_final.index, size=len(X_train_resampled_final), replace=True)
        
        # Create a bootstrapped sample by indexing into the column with the selected indices for the training set
        train_sample = X_train_resampled_final.loc[train_sample_indices, col_name]
        
        # Append the bootstrapped samples to the list for the training set
        train_bootstrapped_samples.append(train_sample)
        
        # Calculate the right trimmed mean of the bootstrapped sample for the training set
        train_right_trimmed_mean = np.mean(train_sample[train_sample <= np.percentile(train_sample, 100*(1-trim_props[col_name]))])
        train_trimmed_means_list.append(train_right_trimmed_mean)
        
    # Calculate the mean of the right trimmed means for the training set and add it to the dictionary
    train_trimmed_means[col_name] = np.mean(train_trimmed_means_list)

    # Replace the outliers in the training set with the trimmed means
    X_train_resampled_final.loc[X_train_resampled_final[col_name] > np.percentile(X_train_resampled_final[col_name], 100*(1-trim_props[col_name])), col_name] = train_trimmed_means[col_name]

# Print the trimmed means for each column separately for the training set
print("Train set trimmed means: ", train_trimmed_means)

Train set trimmed means:  {'amount': 141515.64, 'oldbalanceOrg': 67364.61, 'newbalanceOrig': 4441.342, 'oldbalanceDest': 144455.98, 'newbalanceDest': 230238.02}


## PCA

In [25]:
from sklearn.decomposition import PCA
import numpy as np

# assuming X_train and X_test are your training and test data matrices
# standardize the data using the mean and std from the training set
X_train_mean = np.mean(X_train_resampled_final, axis=0)
X_train_std = np.std(X_train_resampled_final, axis=0)
X_train_std[X_train_std == 0] = 1 # avoid division by zero
X_train_std_inv = 1 / X_train_std

X_train_stdized = (X_train_resampled_final - X_train_mean) * X_train_std_inv
X_test_stdized = (X_test - X_train_mean) * X_train_std_inv

# compute the covariance matrix for the training data
cov_matrix_train = np.cov(X_train_stdized.T)

# compute the eigenvectors and eigenvalues for the training data
eig_vals_train, eig_vecs_train = np.linalg.eig(cov_matrix_train)

# select the top k eigenvectors for the training data
pca_train = PCA(n_components=3)
X_train_pca = pca_train.fit_transform(X_train_stdized)

# project the test data onto the selected eigenvectors from the training data
X_test_pca = pca_train.transform(X_test_stdized)

In [26]:
feature_names = list(df_sample.columns)

# print the selected features
print("Selected features:")
for i in range(pca_train.n_components_):
    # find the index of the maximum absolute value in the ith row of the components array
    idx = np.argmax(np.abs(pca_train.components_[i]))
    # print the name of the feature with the maximum absolute value in the ith row of the components array
    print(f"PC{i+1}: {feature_names[idx]}")

Selected features:
PC1: newbalanceDest
PC2: step
PC3: newbalanceOrig


## Hyperparameter tuning

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
import numpy as np

# assuming X_train and y_train are your training data and labels
# define your hyperparameter search space
param_dist = { 
    'n_estimators': sp_randint(100, 250),
    'max_depth': sp_randint(2, 10),
    'learning_rate': np.logspace(-3, 0, num=100),
    'subsample': np.linspace(0.5, 1, num=50),
    'colsample_bytree': np.linspace(0.5, 1, num=50),
}

In [ ]:
# create a LightGBM classifier object
clf = LGBMClassifier(random_state=42)

In [ ]:
# define your RandomizedSearchCV object
random_search = RandomizedSearchCV(
    clf, param_distributions=param_dist,
    n_iter=30, cv=5, scoring='roc_auc', random_state=42
)

In [ ]:
# fit the randomized search object to your training data
random_search.fit(X_train_pca, y_train_resampled_final)

In [ ]:
# print the best hyperparameters found by the search
print("Best hyperparameters: ", random_search.best_params_)

## Model Training

In [ ]:
import lightgbm as lgb
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold

# Define the LightGBM classifier
clf = lgb.LGBMClassifier(num_leaves=50,max_depth=6,)

# Define the cross-validation method
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Iterate over each fold
for fold, (train_index, test_index) in enumerate(kfold.split(X_train_selected, y_train_resampled_final)):
    # Split the data into training and testing sets
    X_fold_train, y_fold_train = X_train.iloc[train_index], y_train.iloc[train_index]
    X_fold_test, y_fold_test = X_train.iloc[test_index], y_train.iloc[test_index]

    # Train the LightGBM classifier
    clf.fit(X_fold_train, y_fold_train)

    # Evaluate the performance of the model on the testing data
    y_pred = clf.predict(X_fold_test)
    report = classification_report(y_fold_test, y_pred)
    print(f"Fold {fold}:")
    print(f"Classification report:\n{report}")